<a href="https://colab.research.google.com/github/AhmedOssamaAhmed/disaster-tweets-LSTM/blob/main/Predicting_disaster_tweets_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1- Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk,csv,numpy 
from nltk import sent_tokenize, word_tokenize, pos_tag
reader = csv.reader(open('/content/drive/MyDrive/predictiong disaster tweet/train.csv', 'r'), delimiter= ",")



tweets = []
target = []
location =[]
keyword = []
for i in reader:
  tweets.append(i[3])
  target.append(i[4])
  keyword.append(i[1])
  location.append(i[2])


print(f" tweets count : {len(tweets)}")
print(f" keyword count : {len(keyword)}")
print(f" location count : {len(location)}")
print(f" target count : {len(target)}")


2- Check head and info of the data

In [ ]:
print(keyword)
print(location)
print(tweets)
print(target)
keyword.pop(0)
location.pop(0)
tweets.pop(0)
target.pop(0)
print(keyword)
print(location)
print(tweets)
print(target)



3- Is there a missing data [how many and the percentage if there]?

In [ ]:
def missing_percentage(data):
  total_data = len(data)
  for j in range(10):
    for i in data:
      if i == "":
        data.remove(i)
  data_no_missing = len(data)
  return ((7614 - data_no_missing)/7614)*100



In [ ]:
print(f"the percentage of missing keywords is {missing_percentage(keyword)}")
print(f"the percentage of missing location is {missing_percentage(location)}")
print(f"the percentage of missing tweets is {missing_percentage(tweets)}")
print(f"the percentage of missing target is {missing_percentage(target)}")

the percentage of missing keywords is 0.8142894667717363 %

the percentage of missing location is 33.28079852902548 %

the percentage of missing tweets is 0.013133701076963489 %

the percentage of missing target is 0.013133701076963489 %


4- How many data in each class?

In [ ]:
print(target)
real_disaster_counter = 0
not_disaster = 0
for i in target:
  if i == '1':
    real_disaster_counter +=1
  elif i == '0':
    not_disaster +=1
print(f"the count of real disaster class is {real_disaster_counter}")
print(f"the count of not disaster class is {not_disaster}")

5- Get the top 15 locations of the data

In [ ]:
def top_15(data):
  freq_dict = {}
  for i in data:
    if i not in freq_dict:
      freq_dict[i] = 1
    else:
      freq_dict[i] +=1

  sorted_dict = dict(sorted(freq_dict.items(), key=lambda item: item[1], reverse = True))
  top_15_items = []
  for i in range(15):
    top_15_items.append(list(sorted_dict.keys())[i])

  return top_15_items

In [ ]:
top_15_locations = top_15(location)
print(top_15_locations)


6- Get the top 15 keyword in the data

In [ ]:
top_15_keywords = top_15(keyword)
print(top_15_keywords)

7- What are the most common words?

  8- What are the most common stop words?

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
import string
print(string.punctuation)
stop_words = stopwords.words('english')
print(stop_words)

In [ ]:
import re
def cleanText(text):
    text=re.sub(r'#[a-zA-Z0-9|_]*','',text)#  remove hashtage 
    text=re.sub(r'@[a-zA-Z0-9|_]*','',text)# remove user name 
    text=re.sub(r'https?:\/\/\S+','',text)#remove hyperlink 
    text=re.sub(r'\W',' ',text) # remove emotions
    text=re.sub('\d+','',text) # remove digits
    text=re.sub(r'^\s+','',text)# remove space in front of text 
    text=re.sub(r'\s+$','',text) #remove space in tail text
    return text

In [ ]:
tweets_copy = tweets
result = []
for i in tweets_copy:
  tweet=cleanText(i)
  tweet=tweet.split()
  result.append(tweet)


In [ ]:
common_dict={}
for i in range(len(result)):
  for j in range(len(result[i])):
    if result[i][j] not in common_dict:
      common_dict[result[i][j]] = 1
    else:common_dict[result[i][j]] += 1

sorted_dict = dict(sorted(common_dict.items(), key=lambda item: item[1], reverse = True))
top_5_items = []
for i in range(5):
  top_5_items.append(list(sorted_dict.keys())[i])
print(top_5_items)

the most common stop words are :
['the', 'a', 'to', 'in', 'of']

In [ ]:
from nltk.stem import PorterStemmer


def process_tweets(tweets):
    result=[]
    for tweet in tweets:

        tweet=cleanText(tweet)
        tweet=tweet.split()
        #remove stop words 
        tweet=[word for word in tweet  if word.lower() not in stop_words]
        #stemming 
        ps=PorterStemmer()

        tweet=[ps.stem(word) for word in tweet ]

        result.append(tweet)
    return result 

In [ ]:
tweets_processed = process_tweets(tweets)
print(tweets_processed)


In [ ]:
common_words = {}
for i in range(len(tweets_processed)):
  for j in range(len(tweets_processed[i])):
    if tweets_processed[i][j] not in common_words:
      common_words[tweets_processed[i][j]] = 1
    else:common_words[tweets_processed[i][j]] +=1
sorted_dict = dict(sorted(common_words.items(), key=lambda item: item[1], reverse = True))
top_5_items = []
for i in range(5):
  top_5_items.append(list(sorted_dict.keys())[i])
print(top_5_items)

the most common words are :['like', 'fire', 'û_', 'amp', 'get']



9- Use nlp to prepare dataset [tokenization, pad sequence, etc.]

In [ ]:
print(tweets_processed)

Padding

In [ ]:
max = 0
min = 900
for i in tweets_processed:
  if len(i) > max:
    max = len(i)
  if len(i) < min:
    min = len(i)
print(max,min)

In [ ]:
len_words = []
for i in tweets_processed:
  len_words.append(len(i))
sum = 0
for j in len_words:
  sum = sum + j
avg_len = int(sum/len(len_words))

avg_len

In [ ]:
# for tweet in tweets_processed:
#   if len(tweet) > avg_len:
#     for k in range(len(tweet)-avg_len):
#       tweet.pop(0)
#   elif len(tweet) < avg_len:
#     for j in range(avg_len - len(tweet)):
#       tweet.append("o")


In [ ]:
max = 0
min = 900
for i in tweets_processed:
  if len(i) > max:
    max = len(i)
  if len(i) < min:
    min = len(i)
print(max,min)

In [ ]:
print(tweets_processed)
tweets_lst = []
for tweet in tweets_processed:
    lst = ""
    for word in tweet:
        lst += word
        lst += " "
    tweets_lst.append(lst)
print(tweets_lst)

In [ ]:
import pandas as pd
tweets_df = pd.DataFrame(tweets_lst,columns = ["tweet"])
tweets_df.head()

In [ ]:
tweets_df["sentiment"] = target
tweets_df.head()

In [ ]:
print(tweets_df)

10- Prepare train, test sets

In [ ]:
print(len(tweets_df))
print(tweets_df.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(tweets_df,test_size=0.2,random_state=0,shuffle=True)


In [ ]:
print(X_test)

In [ ]:
label_to_sentiment = {"0":"Negative", "1":"Positive"}
def mapper(label):
     return label_to_sentiment[label]
X_train.sentiment = X_train.sentiment.apply(lambda x: mapper(x))
X_test.sentiment = X_test.sentiment.apply(lambda x: mapper(x))

X_train.head()

In [ ]:
print(len(X_train))
print(len(X_test))


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(X_train.tweet)
word_index = tokenizer.word_index
print(word_index)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
# The tokens are converted into sequences and then passed to the pad_sequences() function
x_train = pad_sequences(tokenizer.texts_to_sequences(X_train.tweet),maxlen = 8)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test.tweet),maxlen = 8)

In [ ]:
labels = ['Negative', 'Positive']
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(X_train.sentiment.to_list())
y_train = encoder.transform(X_train.sentiment.to_list())
y_test = encoder.transform(X_test.sentiment.to_list())
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

11- Train your LSTM structure

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
import numpy as np
embeddings_index = {}
# opening the downloaded glove embeddings file
f = open('glove.6B.300d.txt')
for line in f:
    # For each line file, the words are split and stored in a list
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' %len(embeddings_index))

In [ ]:
# creating an matrix with zeroes of shape vocab x embedding dimension
embedding_matrix = np.zeros((vocab_size, 300))
# Iterate through word, index in the dictionary
for word, i in word_index.items():
    # extract the corresponding vector for the vocab indice of same word
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Storing it in a matrix
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf
embedding_layer = tf.keras.layers.Embedding(vocab_size,300,weights=[embedding_matrix],
                                          input_length=8,trainable=False)

In [ ]:
# Import various layers needed for the architecture from keras
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
# The Input layer 
sequence_input = Input(shape=(8,), dtype='int32')
# Inputs passed to the embedding layer
embedding_sequences = embedding_layer(sequence_input)
# dropout and conv layer 
x = SpatialDropout1D(0.2)(embedding_sequences)
x = Conv1D(64, 5, activation='relu')(x)
# Passed on to the LSTM layer
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
# Passed on to activation layer to get final output
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
LR =3e-4
model.compile(optimizer=Adam(learning_rate=LR), loss='binary_crossentropy',metrics=['accuracy'])
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,min_lr = 0.01, monitor = 'val_loss',verbose = 1)

In [ ]:
import keras
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                  restore_best_weights=True,
                                                 monitor="val_accuracy")
training = model.fit(x_train, y_train, batch_size=256, epochs=50,
                    validation_data=(x_test, y_test), callbacks=[ReduceLROnPlateau,early_stopping_cb])

12- Evaluate the model and make predictions

In [ ]:
def predict_tweet_sentiment(score):
    return "Positive" if score>0.5 else "Negative"
scores = model.predict(x_test, verbose=1, batch_size=10000)
model_predictions = [predict_tweet_sentiment(score) for score in scores]
print(model_predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(list(X_test.sentiment), model_predictions))

13- Evaluate the results with charts of acc and loss

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

14- Save your model

In [ ]:
model.save('saved_model/my_model')

test data

In [ ]:
reader_test_final = csv.reader(open('/content/drive/MyDrive/predictiong disaster tweet/test.csv', 'r'), delimiter= ",")
tweets_test=[]
for j in reader_test_final:
    tweets_test.append(j[3])
print(tweets_test)
tweets_test.pop(0)
tweets_copy = tweets_test
result = []
tweets_processed = process_tweets(tweets)
print(tweets_processed)
tweets_lst = []
for tweet in tweets_processed:
    lst = ""
    for word in tweet:
        lst += word
        lst += " "
    tweets_lst.append(lst)
print(tweets_lst)
import pandas as pd
tweets_df = pd.DataFrame(tweets_lst,columns = ["tweet"])
tweets_df.head()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets_df.tweet)
word_index = tokenizer.word_index
print(word_index)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)
test = pad_sequences(tokenizer.texts_to_sequences(tweets_df.tweet),maxlen = 8)

scores = model.predict(test, verbose=1, batch_size=10000)
model_predictions_final = [predict_tweet_sentiment(score) for score in scores]
print(model_predictions_final)